In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
from data import *

/anaconda3/envs/python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [43]:
df = pd.read_csv('data/beijing_201802_201803_aq.csv')
bj_stations = list(df.groupby(by='stationId').size().index)

In [66]:
bj_stations

['aotizhongxin_aq',
 'badaling_aq',
 'beibuxinqu_aq',
 'daxing_aq',
 'dingling_aq',
 'donggaocun_aq',
 'dongsi_aq',
 'dongsihuan_aq',
 'fangshan_aq',
 'fengtaihuayuan_aq',
 'guanyuan_aq',
 'gucheng_aq',
 'huairou_aq',
 'liulihe_aq',
 'mentougou_aq',
 'miyun_aq',
 'miyunshuiku_aq',
 'nansanhuan_aq',
 'nongzhanguan_aq',
 'pingchang_aq',
 'pinggu_aq',
 'qianmen_aq',
 'shunyi_aq',
 'tiantan_aq',
 'tongzhou_aq',
 'wanliu_aq',
 'wanshouxigong_aq',
 'xizhimenbei_aq',
 'yanqin_aq',
 'yizhuang_aq',
 'yongdingmennei_aq',
 'yongledian_aq',
 'yufa_aq',
 'yungang_aq',
 'zhiwuyuan_aq']

In [65]:
df = pd.read_csv('sample_submission.csv')
ld_stations = list(set([x.split('#')[0] for x in list(df['test_id'])]) - set(bj_stations))

In [95]:
def predict(city='bj', metrics=['PM2.5'], station_names=['aotizhongxin_aq'],
            date_to_forc=pd.Timestamp('2018-05-01 00:00:00'), changepoint_prior_scale=10):
    df = airQualityData(city, date_to_forc - pd.Timedelta(7, unit='d'),
                        date_to_forc - pd.Timedelta(1, unit='h'))
    submit_df = pd.DataFrame()
    for station in station_names:
        df_station = pd.DataFrame()
        df_station['test_id'] = [station + '#' + str(i) for i in range(0, 48)]
        for metric in metrics:
            try:
                metric_df = df[df['station_id'] == station][['time', metric]]
                metric_df.columns = ['ds', 'y']
                metric_df = metric_df.reset_index(drop=True)
                metric_df['y'] = np.log(metric_df['y'])

                #metric_df['cap'] = 500 if metric == 'PM2.5' else 1000
                #metric_df['floor'] = 10
                m = Prophet(#growth='logistic', # default: linear
                            yearly_seasonality=False, weekly_seasonality=False, daily_seasonality='auto',
                            #seasonality_prior_scale=0.1, # no use if all seasonality set to False
                            changepoint_prior_scale=changepoint_prior_scale)
                m.fit(metric_df)
                future = m.make_future_dataframe(periods=72, freq='H', include_history=True)
                #future['cap'] = 500 if metric == 'PM2.5' else 1000
                #future['floor'] = 10
                forecast = m.predict(future)
                #m.plot(forecast)
                forecast = forecast[(forecast['ds'] >= date_to_forc)
                                    & (forecast['ds'] < date_to_forc + pd.Timedelta(2, unit='d'))]
    #             print(forecast)
                df_station[metric] = list(np.exp(forecast['yhat']))
    #             print(df_station)
            except:
                df_station[metric] = list(np.random.random((48)) * 50)
        submit_df = pd.concat([submit_df, df_station])
    return submit_df

In [96]:
bj_pred = predict('bj', ['PM2.5', 'PM10', 'O3'], bj_stations)

/anaconda3/envs/python36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [97]:
bj_pred

,test_id,PM2.5,PM10,O3
0,aotizhongxin_aq#0,8.208185,29.284391,15.608697
1,aotizhongxin_aq#1,9.822651,33.836834,30.611591
2,aotizhongxin_aq#2,11.314135,36.072073,51.704015
3,aotizhongxin_aq#3,11.845946,35.896540,66.092213
4,aotizhongxin_aq#4,11.301102,34.519150,67.703937
5,aotizhongxin_aq#5,10.476403,32.614965,64.276268
6,aotizhongxin_aq#6,10.204406,30.050039,62.395329
7,aotizhongxin_aq#7,10.772030,26.951958,61.844295
8,aotizhongxin_aq#8,11.860783,24.277779,59.748342
9,aotizhongxin_aq#9,12.675713,23.130344,55.758269


In [98]:
bj_pred.to_csv('bj_pred.csv', index=None)

In [99]:
ld_pred = predict('ld', ['PM2.5', 'PM10'], ld_stations)

/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log
  
/anaconda3/envs/python36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 0.0.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 0.0.
INFO:fbprophet.forecaster:n_chang

In [100]:
ld_pred

,test_id,PM2.5,PM10
0,GN0#0,10.044040,11.807476
1,GN0#1,8.755956,11.437203
2,GN0#2,8.888312,11.112756
3,GN0#3,10.469900,10.389624
4,GN0#4,12.752984,9.392050
5,GN0#5,14.110348,8.710124
6,GN0#6,13.652692,8.817065
7,GN0#7,12.458458,9.789506
8,GN0#8,12.006369,11.226715
9,GN0#9,12.860293,12.378142


In [101]:
ld_pred.to_csv('ld_pred.csv', index=None)

In [103]:
df = pd.concat([bj_pred, ld_pred])

In [105]:
submit = df.reindex(columns=['test_id', 'PM2.5', 'PM10', 'O3'])

In [106]:
submit.to_csv('submit.csv', index=None)